#### Data import

In [1]:
import pandas as pd

train_df = pd.read_csv('../data/cleaned_train.csv')
test_df = pd.read_csv('../data/cleaned_test.csv')

#### Model import
I tried XGBoost and LGBM

In [2]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error

Checking dataframe

In [3]:
train_df

,season,holiday,workingday,weather,temp,humidity,windspeed,count,date,hours
0,1,0,0,1,9.84,81,0.0000,16,0,0
1,1,0,0,1,9.02,80,0.0000,40,0,1
2,1,0,0,1,9.02,80,0.0000,32,0,2
3,1,0,0,1,9.84,75,0.0000,13,0,3
4,1,0,0,1,9.84,75,0.0000,1,0,4
...,...,...,...,...,...,...,...,...,...,...
8603,3,0,1,1,33.62,49,11.0014,505,565,20
8604,3,0,1,3,27.06,89,16.9979,332,565,21
8605,3,0,1,3,27.06,89,16.9979,68,565,22
8606,3,0,1,3,27.06,89,15.0013,65,565,23


I tried oHe with season but the score was worst

In [60]:
xgb = XGBRegressor(alpha = 10, gamma = 5, learning_rate = 0.05, n_estimators = 200)

Pretty low learning rate, i tried different number a estimators and 200 was generally the best, alpha at 10 : L1 regularization term on weights,
Gamma at 5 : specifies the minimum loss reduction required to make a split.

#### Train, test split

In [61]:
y_train = train_df['count']
X_train = train_df.drop('count', axis=1)
y_test = test_df['count']
X_test = test_df.drop('count', axis=1)

#### XGBoost Scores

In [76]:
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

train_score = xgb.score(X_train, y_train)
r2 = r2_score(y_test, pred)
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = mean_squared_error(y_test, pred, squared=True)
mape = mean_absolute_percentage_error(y_test, pred)

print(f'Train r2 score: ', train_score, '\n'
      'Test: ', '\n'
      ' - r2 score: ',  r2, '\n'
      ' - mae: ',  mae, '\n'
      ' - mse: ',  mse, '\n'
      ' - rmse: ',  rmse, '\n'
      ' - mape: ',  mape, '\n'
      )

Train r2 score:  0.9693490775973694 
Test:  
 - r2 score:  0.9073174669582437 
 - mae:  45.10766438572958 
 - mse:  4401.119458861249 
 - rmse:  4401.119458861249 
 - mape:  0.37048909309619144 



In [77]:
from sklearn.model_selection import learning_curve, StratifiedKFold, cross_val_score
import numpy as np

skf = StratifiedKFold(n_splits=10)
print(f'Cross val score: ', np.mean(cross_val_score(xgb, X_train, y_train, cv=skf)))

C:\Users\Apprenant\anaconda3\envs\datascience\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Cross val score:  0.9433874577508565


In [78]:
import matplotlib.pyplot as plt


N, train_score, val_score = learning_curve(xgb, X_train, y_train, train_sizes = np.linspace(0.1,1,50), cv=skf, scoring='r2')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.plot(N, train_score.mean(axis=1), label='train')

plt.xlabel('train_sizes')
plt.title('Xgb learning curve')
plt.legend()
plt.show() 

C:\Users\Apprenant\anaconda3\envs\datascience\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [70]:
import pickle
pickle.dump(xgb, open('xgb_best.pkl', 'wb'))